In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def canny(image):
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)#0 is for the deviation
    canny=cv2.Canny(blur,50,150)#lower threshold of 50 and upper threshold of 150
    return canny

def region_of_interest(image):
    height=image.shape[0]
    polygons=np.array([[(200,height),(1100,height),(550,250)]])
    #this polygon we are gonna apply on a black mask that has the same dimensions as our input image
    mask=np.zeros_like(image)#completely black as all the pixels are of zero intensity
    cv2.fillPoly(mask,polygons,255)#255 for color specification i.e white and modifies the mask
    #fillPoly fills an array of polygons, so here we want only one polygon so put it as a array of a single element i.e triangle
    masked_image=cv2.bitwise_and(image,mask)
    return masked_image

def display_lines(image,lines):
    line_image=np.zeros_like(image)
    if lines is not None:
        for x1,y1,x2,y2 in lines:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)#takes arg,image,point1,point2,color_in_BGR,thickness_in_pixels
    return line_image

def make_coordinates(image,line_parameters):
    slope,intercept=line_parameters
    #start plotting line from the bottom of the image
    y1=image.shape[0]
    #plot upto 3/5th of the height of the image
    y2=int(y1*(3/5))
    #determine x1 and x2 from the line equation y=mx+b
    x1=int((y1-intercept)/slope)
    x2=int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])

def average_slope_intercept(image,lines):
    #first split the lines into left and right
    left_fit=[]
    right_fit=[]
    for line in lines:
        x1,y1,x2,y2=line.reshape(4)
        parameters=np.polyfit((x1,x2),(y1,y2),1)
        slope=parameters[0]
        y_intercept=parameters[1]
        if slope<0:
            left_fit.append((slope,y_intercept))
        else:
            right_fit.append((slope,y_intercept))
    #now average out the left ant right side parameter values
    left_fit_average=np.average(left_fit,axis=0)
    right_fit_average=np.average(right_fit,axis=0)
    #if both the left anf right fits are non-empty sets plot the lines
    if len(left_fit) and len(right_fit):
        left_fit_average  = np.average(left_fit, axis=0)
        right_fit_average = np.average(right_fit, axis=0)
        left_line  = make_coordinates(image, left_fit_average)
        right_line = make_coordinates(image, right_fit_average)
        averaged_lines = [left_line, right_line]
        return averaged_lines
    
cap = cv2.VideoCapture("test2.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        canny_img = canny(frame)
        cropped_image = region_of_interest(canny_img)
        lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40,maxLineGap=5)
        average_lines = average_slope_intercept(frame, lines)
        line_image = display_lines(frame, average_lines)
        combined_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
        cv2.imshow("result", combined_image)
        if cv2.waitKey(1) == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()